<a href="https://colab.research.google.com/github/dcheung11/chess.com-calculator-python/blob/main/chess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chess.com

from chessdotcom import get_leaderboards, get_player_stats, get_player_game_archives
import pprint
import requests
import numpy as np
printer = pprint.PrettyPrinter()


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from functools import WRAPPER_ASSIGNMENTS

def get_monthly_games(archive):
    response = requests.get(archive)
    if response.status_code == 200:
        data = response.json()
        return data['games']
        # do something with the data here
    else:
        print(f'Request failed with status code {response.status_code}')
        return

def get_game_archives(username):
    all_games = []
    data = get_player_game_archives(username).json

    for archive in data["archives"][-12:]:
        monthly_games = get_monthly_games(archive)
        all_games.extend(monthly_games)
    return all_games

def get_top_wins(games,username):
    wins =0
    top_wins = {}
    for game in games:
        is_white_winner = game["white"]["username"] == username  and game["white"]["result"] == 'win'
        is_black_winner = game["black"]["username"] == username and game["black"]["result"] == 'win'
        if is_white_winner:
            wins += 1
            top_wins[game['black']['username']] = game['black']['rating']
        elif is_black_winner:
            wins += 1
            top_wins[game['white']['username']] = game['white']['rating']

    # Sort the array in descending order
    sorted_dict = dict(sorted(top_wins.items(), key=lambda x: x[1], reverse=True))
    return sorted_dict
    # printer.pprint(sorted_dict)


In [ ]:
def create_user_path(users, target):
    if target in users:
        if len(users) < 2:
          return "You beat " + users[0]
        path = users[0] + " beat "
        print(users)
        current_user = users[1]

        for user in users[2:]:
            path += current_user + " who beat "
            current_user = user
            if user == target:
                path += user
                return path
        return target + " not found in user list"
    else:
        return target + " not found in user list"

visited = set()
player_found = False

def search_for_player(username, path, max_path, visited_players, target):
    global player_found
    
    if player_found:
        return
    
    if path > max_path:
        print("Max search depth reached")
        return
    
    if username in visited:
        return
    
    visited.add(username)
    
    print("Searching top wins for " + username + " with a target of " + target)
    
    games = get_game_archives(username)
    top_wins = get_top_wins(games, username)
    
    if target in top_wins:
        visited_players.append(target)
        # print("Done in " + str(path))
        # print(" ----beat----> ".join(visited_players))
        print(create_user_path(visited_players, target))
        print("Degrees apart: ", len(visited_players) - 1)

        player_found = True
    else:
        visited_players.append(username)
        # print("Search num " + str(path))
        # print("Top wins for " + username + ": " + str(top_wins)) 
        for key in top_wins.keys():
            search_for_player(key, path+1, max_path, visited_players[:], target)


# search_for_player("damedolla5", 0, 20, [], "MagnusCarlsen")


In [ ]:
def transitive_chess_search(me, target):
  print("Have you beaten " + target + "?.....")
  search_for_player(me, 0, 20, [], target)

# transitive_chess_search("damedolla5", "MagnusCarlsen")
transitive_chess_search("damedolla5", "GothamChess")



Have you beaten GothamChess?.....
Searching top wins for damedolla5 with a target of GothamChess
Searching top wins for Kytan with a target of GothamChess
['damedolla5', 'GothamChess']
GothamChess not found in user list
Degrees apart:  1
